In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import lsst.daf.persistence as dafPersist
from pfs.lam.opdb import *
from pfs.lam.analysisPlot import *
from pfs.lam.detAnalysis import *

import pandas as pd
import os
import re

In [ ]:
experimentId = 125
cam = "b2"
rerun = "defocus" #"dcb2"

peaklist = "../SM2-defocus/Imqual_LAM_peaklist_DCB2-notfinal.csv"

In [ ]:
peaklist = os.environ['LAM_SPS_ANALYSIS_DIR']+"/notebooks/devel/fmadec/SM2-defocus/Imqual_LAM_peaklist_DCB2-notfinal.csv"


In [ ]:
visitStart, visitEnd = getVisitRange(experimentId)
print(visitStart, visitEnd)

In [ ]:
drpPath = "/drp"
repo = "sps"
basePath = '/drp/analysis/sm2/'
arm = cam[0]


In [ ]:
visit = int(visitStart)

In [ ]:
print(f"{drpPath}/{repo}/rerun/{rerun}/detrend/")

butler = dafPersist.Butler(f"{drpPath}/{repo}/rerun/{rerun}/detrend/")
rbutler = dafPersist.Butler(f"{drpPath}/{repo}/")
rbutler.getKeys('raw')
lamps = rbutler.queryMetadata('raw', ['lamps'], visit=visit, arm=arm) 
print(f"lamps used: {lamps}")
calExp = butler.get("calexp", visit=visit, arm=cam[0])

In [ ]:
peaks = pd.read_csv(peaklist)
selected_peaks = peaks[peaks.arm == arm]
selected_peaks = selected_peaks[(selected_peaks.element.str.contains('|'.join(re.findall('[A-Z][^A-Z]*', "".join(lamps)))))]

In [ ]:
plotRoiPeak(calExp.image.array, selected_peaks, raw=True, roi_size=30)

In [ ]:
EE = 20
doPlot = True

In [ ]:
imdata = getImageQuality(calExp.image.array, selected_peaks,roi_size=24, EE=[EE], com=True, doPlot=doPlot)

In [ ]:
dataId = dict(arm="b", spectrograph=2)
dataId.update(visit=int(visitStart))
exp = butler.get("calexp", dataId)
print(dataId)

In [ ]:
f"{dataId['arm']}{dataId['spectrograph']}"

In [ ]:
calexfilePath = butler.getUri("calexp", dataId)
print(calexfilePath)

In [ ]:
imageInfo = dict(dataId)
imageInfo.update(filename=calexfilePath)

In [ ]:
print(dataId)
print(imageInfo)

In [ ]:
df = ImageQualityToCsv(butler, dataId, selected_peaks, "./", doFit=False,doPlot=True)

In [ ]:
exp.getMetadata().toDict()['W_ENFCAX']

In [ ]:
waves = selected_peaks.wavelength.unique()
waves[0]

In [ ]:
p = selected_peaks.groupby(["fiber", "wavelength"]).get_group((2,waves[0]))
cx = p.X
cy = p.Y

In [ ]:
plotOnePeak(calExp.image.array, cx,cy)

In [ ]:
(nx,ny) = estimateCOM(calExp.image.array, cx,cy, roi_size=16, seek_size=50)

In [ ]:
plotOnePeak(calExp.image.array, nx,ny)

In [ ]:
def estimateCOM(image, cx, cy, roi_size=30, doBck=True, nRows=5, seek_size=None):
    """
    estimate center of mass 
    if seek_size is given, recentration will be tried using the seek_size
    if doBck = True it used the rRowns outside of the roi_size to evaluate the bck level and substract it to the roi sub_image
    Return centroid of the center of mass in px
    """
    if seek_size is not None:
        (cx,cy) = estimateCOM(image, cx, cy, roi_size=seek_size, doBck=doBck, nRows=nRows)       
    
    indx = int(cy)
    indy = int(cx)
    half = int(roi_size/2)
    
    outer_data = selectRoi(image, cx, cy, roi_size=roi_size, doBck=doBck, nRows=nRows)            
    y, x = center_of_mass(outer_data)  

    return indy - half + x + 0.5, indx - half + y + 0.5